<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/neo4jdocs/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium==4.7.2 transformers==4.25.1 sentence-transformers===2.2.2 graphdatascience

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt-get update
!apt install chromium-chromedriver
!apt install -y xvfb

!pip install undetected-chromedriver==3.1.7
!pip install PyVirtualDisplay

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [3]:
!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
# replace python3.8 with your own version of python in case it's not the same
patcher_src = (
    "/usr/local/lib/python3.8/dist-packages/undetected_chromedriver/patcher.py"
)
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace(
        "return urlretrieve(u)[0]",
        "return urlretrieve('file:///content/chromedriver_linux64.zip',"
        "filename='/tmp/chromedriver_linux64.zip')[0]",
    )
with open(patcher_src, "w") as f:
    f.write(contents)

updating: chromedriver (deflated 51%)


In [4]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [6]:
def extract_text_by_class(c):
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, c)
        return content.text
    except:
        return ""


def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Remove links to images and various files
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained(
    "yanekyuk/bert-uncased-keyword-extractor"
)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    for token in nlp(text):
        if token['entity'] == 'I-KEY':
            keyword += token['word'][2:] if \
              token['word'].startswith("##") else f" {token['word']}"
        else:
            if keyword:
                result.append(keyword)
            keyword = token['word']
    # Add the last keyword
    result.append(keyword)
    return list(set(result))

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [8]:
extract_keywords(
    """
Broadcom agreed to acquire cloud computing company VMware in a $61 billion (€57bn) cash-and stock deal.
"""
)

['cloud computing', 'vmware', 'broadcom']

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [10]:
generate_embeddings(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

[-0.06911398470401764,
 0.05224500596523285,
 -0.007201771251857281,
 -0.015374748036265373,
 0.003906042082235217,
 -0.050424572080373764,
 -0.10675834119319916,
 0.062257070094347,
 0.0032547430600970984,
 0.013045604340732098,
 0.007432149723172188,
 0.002438630210235715,
 0.07324251532554626,
 0.060171253979206085,
 0.13454286754131317,
 0.036663979291915894,
 -0.007088577374815941,
 0.030180800706148148,
 0.026259878650307655,
 -0.1478315144777298,
 0.011475421488285065,
 0.0016846859361976385,
 -0.046527840197086334,
 -0.03765556588768959,
 0.048804327845573425,
 0.08559446036815643,
 -0.030265992507338524,
 0.052127692848443985,
 0.045675382018089294,
 0.06373857706785202,
 -0.029776496812701225,
 -0.01067135389894247,
 -0.05804385617375374,
 0.058136411011219025,
 -0.09820526838302612,
 0.009667886421084404,
 0.03530015051364899,
 0.00953009631484747,
 0.01339051965624094,
 0.0526607520878315,
 0.04211461916565895,
 0.01219760999083519,
 -0.04744802787899971,
 0.018271863460540

In [11]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

while visit_list:
    # Visit the URL
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
    # If redirect
    actual_url = wd.current_url
    if actual_url != current_url:
        # Store the redirect information
        data[current_url] = {
          "links": [],
          "text": None,
          "embeddings": [],
          "keywords": [],
          "redirects": [actual_url]
        }
        already_visited.append(current_url)
        # Content is being extracted from the actual url
        current_url = actual_url

    # Extract text from the content div
    text = extract_text_by_class("content")
    # If nothing is found, try article div
    if not text:
        text = extract_text_by_class("article")
    # If nothing is found, try page div
    if not text:
        text = extract_text_by_class("page")
    if not text:
        text = extract_text_by_class("single-user-story")
    # Check if 404
    if ("Sorry, page not found" in wd.find_element(By.TAG_NAME, "body").text):
        text = "404"

    # Generate paragraph embedding & extract keywords
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

    # Extract links from the content div
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")
    if not links:
        links = extract_links_by_xpath("//article//a[@href]")

    if not links and not text:
        print(f"Couldn't retrieve the data from {current_url}")
    # Store page information
    data[current_url] = {
        "links": links,
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
        "redirects": []
    }
    # Crawling information
    already_visited.append(current_url)
    # Don't leave neo4j.com while crawling
    # and avoid community and sandbox
    visit_list.extend(
        [l for l in list(links) if ("neo4j.com" in l)  
        and (not l in already_visited)  
        and (not "community.neo4j.com" in l)  
        and (not "sandbox.neo4j.com" in l)]
  )

https://neo4j.com/docs
https://neo4j.com/docs/apoc/current
https://neo4j.com/docs/apoc/current/database-introspection
https://neo4j.com/docs/apoc/5/background-operations/triggers
https://neo4j.com/docs/apoc/5/overview/apoc.trigger/apoc.trigger.resume
https://neo4j.com/docs/apoc/5/overview/apoc.util
https://neo4j.com/docs/apoc/5/overview/apoc.util/apoc.util.sha1
https://neo4j.com/docs/apoc/5/overview/apoc.util/apoc.util.sha256
https://neo4j.com/docs/apoc/5/overview/apoc.util/apoc.util.sha384
https://neo4j.com/docs/apoc/5/overview/apoc.util/apoc.util.sha512
https://neo4j.com/docs/apoc/5/overview/apoc.util/apoc.util.validatePredicate
https://neo4j.com/docs/apoc/5/overview/apoc.xml
https://neo4j.com/docs/apoc/5/overview/apoc.xml/apoc.xml.parse
https://neo4j.com/docs/apoc/5/import/xml
https://neo4j.com/docs/apoc/5/overview/apoc.load/apoc.load.xml
https://neo4j.com/docs/apoc/5/overview/apoc.load/apoc.load.jsonParams
https://neo4j.com/docs/apoc/5/overview/apoc.load/apoc.load.jsonArray
https:/

KeyboardInterrupt: ignored

In [ ]:
import json

j = json.dumps(data)
# open file for writing, "w"
f = open("neo4j_docs.json", "w")

# write json object to file
f.write(j)

# close file
f.close()

In [ ]:
# Import to Neo4j
from graphdatascience import GraphDataScience

host = "bolt://44.200.120.93:7687"
user = "neo4j"
password = "customs-spans-misalinements"

gds = GraphDataScience(host, auth=(user, password))

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
"""
)

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
"""
)

In [ ]:
import_query = """

UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
FOREACH (r in row.redirects| MERGE (r1:Page {url:r}) MERGE (p)-[:REDIRECTS]->(r1))

"""
x = 1
params = []
for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
            "redirects": data[key]["redirects"]
        }
    )
    # Batch per 500
    if len(params) == 500:
        gds.run_cypher(import_query, {"data": params})
        params = []
        # Logging
        print(f"Importing {x} batch")
        x += 1

# Import the remainder
gds.run_cypher(import_query, {"data": params})